# Introduction

In this notebook, we will use the existing pipeline_ex_XGBoost to build and test an XGBoost model to predict the scoreline for EPL games.

# Step 1 - Import and Data Prep

We import the necessary libraries and get the training, validation and testing datasets. These sets have undergone cleaning, preparation and feature engeneering in the original pipeline file, so they are ready to use

In [1]:
# XGBoost Evaluation Notebook

# Imports
import numpy as np
import pandas as pd
from xgboost import XGBRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
import sys, os

# Import pipeline and load data
sys.path.append(os.path.abspath('..'))  # Adjust if needed
from pipeline_ex_XGBoost import get_train_val_test_data

X_train, X_val, X_test, y_train, y_val, y_test = get_train_val_test_data()

c:\Users\nikol\Documents\ML\ML_group_project\pipeline_ex_XGBoost.py:190: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  db['odds_hw'] = db[home_win_cols].mean(axis=1)
c:\Users\nikol\Documents\ML\ML_group_project\pipeline_ex_XGBoost.py:192: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  db['odds_d']  = db[draw_cols].mean(axis=1)
c:\Users\nikol\Documents\ML\ML_group_project\pipeline_ex_XGBoost.py:194: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor 

--- NaN Check before scaling ---
NaNs in X_train: 0
NaNs in X_val: 0
NaNs in X_test: 0
--- NaN Check after scaling ---
NaNs in X_train_scaled: 0
NaNs in X_val_scaled: 0
NaNs in X_test_scaled: 0


c:\Users\nikol\Documents\ML\ML_group_project\pipeline_ex_XGBoost.py:159: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  db["h2h_avg_home_goals_last_5"].fillna(db["h2h_avg_home_goals_last_5"].mean(), inplace=True)
c:\Users\nikol\Documents\ML\ML_group_project\pipeline_ex_XGBoost.py:160: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which 

In [2]:
print("X_train type:", type(X_train))
print("X_train shape:", X_train.shape)
print("y_train type:", type(y_train))
print("y_train shape:", y_train.shape)
print("First 5 y_train columns:", y_train.columns.tolist())
print("First 5 X_train columns:", X_train.columns.tolist())

X_train type: <class 'pandas.core.frame.DataFrame'>
X_train shape: (3233, 227)
y_train type: <class 'pandas.core.frame.DataFrame'>
y_train shape: (3233, 2)
First 5 y_train columns: ['FTHG', 'FTAG']
First 5 X_train columns: ['HTHG', 'HTAG', 'HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC', 'HY', 'AY', 'HR', 'AR', 'Bb1X2', 'BbMxH', 'BbAvH', 'BbMxD', 'BbAvD', 'BbMxA', 'BbAvA', 'BbOU', 'BbMx_2.5', 'BbAv_2.5', 'BbMx_2.5', 'BbAv_2.5', 'BbAH', 'BbAHh', 'BbMxAHH', 'BbAvAHH', 'BbMxAHA', 'BbAvAHA', 'B365_2.5', 'B365_2.5', 'P_2.5', 'P_2.5', 'Max_2.5', 'Max_2.5', 'Avg_2.5', 'Avg_2.5', 'AHh', 'B365AHH', 'B365AHA', 'PAHH', 'PAHA', 'MaxAHH', 'MaxAHA', 'AvgAHH', 'AvgAHA', 'B365C_2.5', 'B365C_2.5', 'PC_2.5', 'PC_2.5', 'MaxC_2.5', 'MaxC_2.5', 'AvgC_2.5', 'AvgC_2.5', 'AHCh', 'B365CAHH', 'B365CAHA', 'PCAHH', 'PCAHA', 'MaxCAHH', 'MaxCAHA', 'AvgCAHH', 'AvgCAHA', 'BFEH', 'BFED', 'BFEA', 'BFE_2.5', 'BFE_2.5', 'BFEAHH', 'BFEAHA', 'BFECH', 'BFECD', 'BFECA', 'BFEC_2.5', 'BFEC_2.5', 'BFECAHH', 'BFECAHA', 'odds_hw

# Step 2 - Hyperparameter Tuning

We tune the hyperparemeters using GridSearchCV to get the optimal model performance. Due to a bug in recent versions of XGBoost, we have to pass both the features and targets as numpy arrays to make the process work,

In [3]:
param_grid = {
    'estimator__n_estimators': [50, 100, 200],
    'estimator__max_depth': [3, 5, 7],
    'estimator__learning_rate': [0.01, 0.1, 0.2],
    'estimator__subsample': [0.8, 1.0],
    'estimator__colsample_bytree': [0.8, 1.0]
}

# Create time series cross-validation splits
tscv = TimeSeriesSplit(n_splits=5)

base_model = MultiOutputRegressor(XGBRegressor(objective='reg:squarederror', random_state=42, n_jobs=-1))
grid = GridSearchCV(base_model, param_grid, cv=tscv, scoring='neg_root_mean_squared_error', verbose=2, n_jobs=-1)
grid.fit(X_train.values, y_train.values)

print("Best parameters:", grid.best_params_)
print("Best CV score (neg RMSE):", grid.best_score_)

Fitting 5 folds for each of 108 candidates, totalling 540 fits
Best parameters: {'estimator__colsample_bytree': 1.0, 'estimator__learning_rate': 0.1, 'estimator__max_depth': 3, 'estimator__n_estimators': 50, 'estimator__subsample': 1.0}
Best CV score (neg RMSE): -0.8234924793243408


# Step 4 - Validation Evaluation

We validate the model and see how it performs on the validation set. This should give us an idea of how it performs and if the hyperparameter optimization was successful

In [4]:
# Validation set evaluation with rounded predictions
y_val_pred = grid.predict(X_val.values)
y_val_pred_rounded = np.round(y_val_pred)  # Round predictions to nearest integer

print("Validation RMSE (raw):", mean_squared_error(y_val, y_val_pred))
print("Validation RMSE (rounded):", mean_squared_error(y_val, y_val_pred_rounded))
print("Validation MAE (raw):", mean_absolute_error(y_val, y_val_pred))
print("Validation MAE (rounded):", mean_absolute_error(y_val, y_val_pred_rounded))
print("Validation R2 (raw):", r2_score(y_val, y_val_pred))
print("Validation R2 (rounded):", r2_score(y_val, y_val_pred_rounded))

Validation RMSE (raw): 0.4681921899318695
Validation RMSE (rounded): 0.535891056060791
Validation MAE (raw): 0.5570856332778931
Validation MAE (rounded): 0.4839108884334564
Validation R2 (raw): 0.6895456314086914
Validation R2 (rounded): 0.6439706087112427


# Step 5 - Retrain and Evaluate

We can now retrain the model on the train+val sets to ensure it sees as many examples as possible while avoiding leakage by leaving the test set untouched. We will use the test set to evaluate performance

In [5]:
# Retrain on train+val, test on test set
X_trainval = pd.concat([X_train, X_val])
y_trainval = pd.concat([y_train, y_val])
final_model = MultiOutputRegressor(
    XGBRegressor(
        objective='reg:squarederror',
        random_state=42,
        n_jobs=-1,
        **{k.replace('estimator__', ''): v for k, v in grid.best_params_.items()}
    )
)
final_model.fit(X_trainval.values, y_trainval.values)
y_test_pred = final_model.predict(X_test.values)

# Step 6 - Evaluation Results

Here we see the results of our testing

In [10]:
# Test set evaluation with rounded predictions
y_test_pred = final_model.predict(X_test.values)
y_test_pred_rounded = np.round(y_test_pred)  # Round predictions to nearest integer

print("Test RMSE (raw):", mean_squared_error(y_test, y_test_pred))
print("Test RMSE (rounded):", mean_squared_error(y_test, y_test_pred_rounded))
print("Test MAE (raw):", mean_absolute_error(y_test, y_test_pred))
print("Test MAE (rounded):", mean_absolute_error(y_test, y_test_pred_rounded))
print("Test R2 (raw):", r2_score(y_test, y_test_pred))
print("Test R2 (rounded):", r2_score(y_test, y_test_pred_rounded))

Test RMSE (raw): 0.39769604802131653
Test RMSE (rounded): 0.4864197373390198
Test MAE (raw): 0.5136377215385437
Test MAE (rounded): 0.44691359996795654
Test R2 (raw): 0.737667441368103
Test R2 (rounded): 0.6786786317825317


We can see comparisons of predictions and actual values to see how our model performed

In [7]:
# Show predictions vs actuals with rounded predictions
results = pd.DataFrame({
    'FTHG_true': y_test['FTHG'].values,
    'FTHG_pred': y_test_pred[:, 0],
    'FTHG_pred_rounded': y_test_pred_rounded[:, 0],
    'FTAG_true': y_test['FTAG'].values,
    'FTAG_pred': y_test_pred[:, 1],
    'FTAG_pred_rounded': y_test_pred_rounded[:, 1]
})
print(results.head())

   FTHG_true  FTHG_pred  FTHG_pred_rounded  FTAG_true  FTAG_pred  \
0          1   1.425803                1.0          0   1.144145   
1          5   4.618654                5.0          0   0.700376   
2          4   2.416402                2.0          2   1.422171   
3          0   0.175415                0.0          0   0.749795   
4          0   0.521366                1.0          2   1.449654   

   FTAG_pred_rounded  
0                1.0  
1                1.0  
2                1.0  
3                1.0  
4                1.0  
